In [10]:
import numpy as np
import array as myarray
from sympy import *
import os

infile= open("aa_code_x3.txt", 'r')
infile.readline() # skip the first line

AA_q = []
AA_eps = []
AA_name = []
AA_num = []

for line in infile:
    words = line.split()
    # words[2]: AA_eps, words[3]: AA_q, words[4]: AA_name
    AA_num.append(words[1])
    AA_eps.append(words[2])
    AA_q.append(words[3])
    AA_name.append(words[4])

infile.close()
AA_eps = AA_eps[:]  #the slice [:-1] picks all except the last element, CYS.
AA_q = AA_q[:]
AA_name = AA_name[:]
AA_num = AA_num[:]

pair_eps = []
for i in range(len(AA_eps)):
    for j in range(len(AA_eps)):
        if i<=j:
            pair_eps.append((AA_eps[i], AA_eps[(j)]))
###print("The pair list for epsilon:", pair_eps, len(pair_eps))
print("The pair list for epsilon:", len(pair_eps))
pair_q = []
for i in range(len(AA_q)):
    for j in range(len(AA_q)):
        if i<=j:
            pair_q.append((AA_q[i], AA_q[(j)]))
###print("The pair list for charges:", pair_q, len(pair_q))
print("The pair list for charges:", len(pair_q))
pair_num = []
for i in range(len(AA_num)):
    for j in range(len(AA_num)):
        if i<=j:
            pair_num.append((AA_num[i], AA_num[(j)]))
###print("The pair list for names:", pair_name, len(pair_name))
print("The pair list for names:", len(pair_num))

pair_name = []
for i in range(len(AA_name)):
    for j in range(len(AA_name)):
        if i<=j:
            pair_name.append((AA_name[i], AA_name[(j)]))
print("The pair list for names:", len(pair_name))

print("===============")

#print(pair_eps[180]); print(pair_q[180]); print(pair_num[180]); print(pair_name[180])

#print(len(pair_eps))

The pair list for epsilon: 3
The pair list for charges: 3
The pair list for names: 3
The pair list for names: 3


In [11]:
def potential(pair_eps,pair_q,pair_num, pair_name):

    file_name = 'nbint'
    aa1 = pair_num[0]
    aa2 = pair_num[1]
    A1 = pair_name[0]
    A2 = pair_name[1]
    file = open(f'ff_FGx3/{file_name}_{aa1}_{aa2}_x3.table', "w")
    j=0

        
    """ Headers """
    
    print("# %s-%s non-bonded potential table using Onck FF" %(A1, A2), file = file)
    print(" " , file = file)
    print("Onck_ff", file = file)
    
    r_start = 0.01
    r_end = 15.01
  ##  r_end = 30.01 ###### Old cutoff
  ##  r_end = 6.01 ####### Only repulsion
    bin_size = 0.01
    N_bins = (r_end - r_start) / bin_size
    
    rep_cutoff = 6.0
    print("N \t %d \t R \t %.2f \t %.2f" %(N_bins, r_start, r_end-bin_size), file = file)

    print(" ",  file = file)


    
    """ Electrostatics """
    ld = 10.0                 # Debye length
    qi = float(pair_q[0])              # Charge i
    qj = float(pair_q[1])              # Charge j
    D = 80.0                  # dielectric of water
    _z = 2.5

    r = symbols('r')

    D = 80.0 * (1.0 - (r/_z)**2 * exp(r/_z)/(exp(r/_z)-1.0)**2)

    A =  332.06371

    u_elec = (A*qi*qj/D)*exp(-r/ld) / r

    df_elec = diff(u_elec, r)

    """ LJ-type term """
    alpha = 0.27
    eps_hp = 3.1070746 #  3.1070746 units "13 kJ/N_A" kcal_mol
    eps_rep = 2.3900574 #  2.3900574 units "10 kJ/N_A" kcal_mol

    sigma = 6.0     # Angstrom unit
    eps_i = float(pair_eps[0])
    eps_j = float(pair_eps[1])

    eps_ij = eps_hp*np.sqrt( (eps_i*eps_j)**alpha )

    r6 = (sigma/r)**6
    r8 = (sigma/r)**8

    """ Forces """
    u_lj1 = eps_rep*r8 - eps_ij*(4.0*r6-1.0)/3.0
    df_lj1 = diff(u_lj1,r)

    u_lj2 = (eps_rep-eps_ij) * r8
    df_lj2 = diff(u_lj2,r)

    
    """ writing output """    
    for i in np.arange(r_start,r_end,bin_size):
        F_elec = df_elec.subs(r, i)
        j=j+1

        if i <= sigma:
            F_lj1 = df_lj1.subs(r, i)
            F_tot = F_lj1 + F_elec

            D = 80 * (1- (i/_z)**2 * exp(i/_z)/(exp(i/_z)-1)**2)
            u_elec = (A*qi*qj/D)*exp(-i/ld) / i
            i6 = (sigma/i)**6
            i8 = (sigma/i)**8
            u_lj1 = eps_rep*i8 - eps_ij*(4*i6-1)/3
            u1 = u_lj1 + u_elec

            print("%d \t %.2f \t %.10e \t %0.10e" %(j, i, u1, -F_tot), file = file)

        else:

            F_lj2 = df_lj2.subs(r, i)
            F_tot = F_lj2 + F_elec

            D = 80 * (1- (i/_z)**2 * exp(i/_z)/(exp(i/_z)-1)**2)
            u_elec = (A*qi*qj/D)*exp(-i/ld) / i
            i6 = (sigma/i)**6
            i8 = (sigma/i)**8
            u_lj2 = (eps_rep-eps_ij) * i8
            u2 = u_lj2 + u_elec

            print("%d \t %.2f \t %.10e \t %.10e" %(j, i, u2, -F_tot), file = file)
    file.close()


In [12]:
for i in range(len(pair_eps)):
    potential(pair_eps[i], pair_q[i], pair_num[i], pair_name[i])

#potential(pair_eps[1], pair_q[1], pair_num[1], pair_name[1])